In [1]:
import requests
from pymongo import MongoClient
import pandas as pd
import random

# Create user

In [1771]:
personajes = ['Barack Obama', 'Voldemort', 'Chewbacca', 'Angela Merkel','Margherita Hack','Valentino Rossi', 
              'must4in3', 'Aretha Franklin', 'Marge Simpson', 'Darth Vader', 'Yoda', 'R2-D2', 'John Belushi',
              'Maccio Capatonda', ' Elon Musk', 'Che Guevara', 'Gollum', 'Samwise Gamgee', 'Jon Snow','Arya Stark',
              'Heidi', 'Hermione', 'Pikachu', 'Topolino', 'Berlusconi', 'Greta Thunberg','Hulk', 'Dilan Dog', 
              'Stephen Hopkins','Oprah Winfre', 'Frida Kahlo','Tyrion Lannister', 'Jennifer Aniston']

for personage in personajes:
    res = requests.get(f'http://localhost:3000/user/create/{personage}')

In [1772]:
print(res)

<Response [500]>


In [1773]:
res.json()

{'message': 'Username already exists. Please choose another one!',
 'status': 'api-error'}

In [907]:
# check users in Mongo
dbName = "api_chat_sentiment_analysis"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [908]:
cur = db.user.find({"username":{"$ne": None}})

In [909]:
df = pd.DataFrame(cur)
df.head()

,_id,username
0,5f0af5eb85f7b2e0de5fb17c,Barack Obama
1,5f0af5eb85f7b2e0de5fb17d,Voldemort
2,5f0af5eb85f7b2e0de5fb17e,Chewbacca
3,5f0af5eb85f7b2e0de5fb17f,Angela Merkel
4,5f0af5eb85f7b2e0de5fb180,Margherita Hack


# Create Chat

In [8]:
df["_id"]= df["_id"].astype(str) 

In [9]:
five_chats = ['How I met your mother', 'Dumbledore’s Army', 
              'Tesla SpaceX','Italian restaurants in Madrid','datamad0620']
chats_room = dict()
for i,chat in enumerate(five_chats):   
    chats_room[f'{chat}']= {'user_ids': [random.choice([row for row in df['_id']]) for _ in range(5)]}

In [10]:
for room,user_ids in chats_room.items():
    res = requests.get(f'http://localhost:3000/chat/create/{room}?user_id={user_ids.values()}')

In [11]:
print(res)

<Response [200]>


In [12]:
res.json()

{'chat_users': 3,
 'create chat status': 'Ok',
 'create users status': 'Some user_id did not exist in the database, or you tried insert the same id many times',
 'data': {'_id': {'$oid': '5f0af5f085f7b2e0de5fb1a1'},
  'chat_name': 'datamad0620',
  'users_ids': ['5f0af5eb85f7b2e0de5fb188',
   '5f0af5eb85f7b2e0de5fb199',
   '5f0af5eb85f7b2e0de5fb19b']}}

In [13]:
cur = db.chatItem.find({"chat_name":{"$ne": None}})

In [14]:
df_chat = pd.DataFrame(cur)
df_chat.head()

,_id,chat_name,users_ids
0,5f0af5ef85f7b2e0de5fb19d,How I met your mother,"[5f0af5eb85f7b2e0de5fb191, 5f0af5eb85f7b2e0de5..."
1,5f0af5f085f7b2e0de5fb19e,Dumbledore’s Army,"[5f0af5eb85f7b2e0de5fb193, 5f0af5eb85f7b2e0de5..."
2,5f0af5f085f7b2e0de5fb19f,Tesla SpaceX,"[5f0af5eb85f7b2e0de5fb187, 5f0af5eb85f7b2e0de5..."
3,5f0af5f085f7b2e0de5fb1a0,Italian restaurants in Madrid,"[5f0af5eb85f7b2e0de5fb183, 5f0af5eb85f7b2e0de5..."
4,5f0af5f085f7b2e0de5fb1a1,datamad0620,"[5f0af5eb85f7b2e0de5fb188, 5f0af5eb85f7b2e0de5..."


# Add user in a Chat

In [20]:
df_chat["_id"]= df_chat["_id"].astype(str) 

In [57]:
user_random = random.choice([row for row in df['_id']])
conversation_random = random.choice([row for row in df_chat['_id']])

In [58]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/adduser?user_id={user_random}')

In [59]:
print(res)

<Response [200]>


In [60]:
res.json()

{'chat_name': 'Tesla SpaceX',
 'users_ids': ['5f0af5eb85f7b2e0de5fb187',
  '5f0af5eb85f7b2e0de5fb18f',
  '5f0af5eb85f7b2e0de5fb195',
  '5f0af5eb85f7b2e0de5fb186',
  '5f0af5eb85f7b2e0de5fb19b',
  '5f0af5eb85f7b2e0de5fb198']}

# Add message in chat

kaggle dataset quote
https://www.kaggle.com/coolcoder22/quotes-dataset

In [61]:
quote = pd.read_csv('inputs/quote.csv', encoding = 'latin-1')

In [62]:
quote

,Auther,quote
0,A. A. Milne,"If you live to be a hundred, I want to live to..."
1,A. A. Milne,Promise me you'll always remember: You're brav...
2,A. A. Milne,"Did you ever stop to think, and forget to star..."
3,A. A. Milne,Organizing is what you do before you do someth...
4,A. A. Milne,"Weeds are flowers too, once you get to know them."
...,...,...
36160,Zsa Zsa Gabor,"I know nothing about sex, because I was always..."
36161,Zsa Zsa Gabor,Conrad Hilton was very generous to me in the d...
36162,Zsa Zsa Gabor,The women's movement hasn't changed my sex lif...
36163,Zsa Zsa Gabor,I always liked parties. You meet people; you c...


In [93]:
conversations = []
for _ in range(100):
    conversations.append({random.choice([row for row in df['_id']]): random.choice([row for row in quote['quote']])})

In [94]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [95]:
for dic in conversations:
    for key,values in dic.items():
        res = requests.get(f'http://localhost:3000/chat/{conversation_random}/addmessage?user_id={key}&text={values}')

In [96]:
print(res)

<Response [200]>


In [97]:
res.json()

{'conversation_id': '5f0af5f085f7b2e0de5fb19e',
 'messages': {'text': 'Without deviation progress is not possible.',
  'user_id': '5f0af5eb85f7b2e0de5fb184'}}

# Get messages list from a chat

In [354]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [355]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/list')

In [356]:
print(res)

<Response [200]>


In [359]:
res.json()

{'chat_name': 'Italian restaurants in Madrid',
 'conversation_id': {'$oid': '5f0af5f085f7b2e0de5fb1a0'},
 'messages': [{'message': 'My father had been in the German Diplomatic Service, and although he had been in the United States since World War II and out of diplomacy, it was something that was very much talked about in the family. So when I went to college, it was always my intention to try to get into the Foreign Service.',
   'user_id': '5f0af5eb85f7b2e0de5fb192',
   'username': 'Pikachu'},
  {'message': 'My writing, like everything I do, comes profoundly from my heart. I believe that if you follow your heart you will be successful in one way or another. Old-fashioned as that might sound, the philosophy is true.',
   'user_id': '5f0af5eb85f7b2e0de5fb193',
   'username': 'Topolino'},
  {'message': 'Motivation is the art of getting people to do what you want them to do because they want to do it.',
   'user_id': '5f0af5eb85f7b2e0de5fb183',
   'username': 'Aretha Franklin'},
  {'mess

# Get sentiment analysis

NLP & Text Sentiment Analysis:
​
- [https://www.nltk.org/]
- [https://towardsdatascience.com/basic-binary-sentiment-analysis-using-nltk-c94ba17ae386]
- [https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk]

In [624]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [625]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/sentiment')

In [626]:
res.json()

[{'sentence': "I'm just trying to portray what I find ironic or humorous.",
  'sentiment': {'neg': 0.124, 'neu': 0.661, 'pos': 0.215, 'compound': 0.2732}},
 {'sentence': 'Art knows neither doctrine nor idea; its nature is to show.',
  'sentiment': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}},
 {'sentence': "When you want a thing done, 'Don't do it yourself' is a good motto for Scoutmasters.",
  'sentiment': {'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.4939}},
 {'sentence': "I don't think necessity is the mother of invention.",
  'sentiment': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}},
 {'sentence': 'Invention, in my opinion, arises directly from idleness, possibly also from laziness - to save oneself trouble.',
  'sentiment': {'neg': 0.136, 'neu': 0.704, 'pos': 0.161, 'compound': 0.128}},
 {'sentence': "What's amazing is, if young people understood how doing well in school makes the rest of their life so much interesting, they would be more motivated.",


# Get reccomender analysis

In [1764]:
user_random = random.choice([row for row in df['_id']])

In [1765]:
res = requests.get(f'http://localhost:3000/user/{user_random}/recommend')

In [1766]:
print(res)

<Response [200]>


In [1767]:
res.json()

{'Me': {'Pikachu': 1.0},
 'Recommended users': {'Angela Merkel': 0.5149709740270669,
  'Greta Thunberg': 0.5875626567521308,
  'John Belushi': 0.5667667460459788,
  'Marge Simpson': 0.6085071734461294,
  'Oprah Winfre': 0.5978005331549622}}